In [ ]:
````xml
<!-- filepath: e:\hack\langchain-in-action\notebooks\chapter01\ch01_01_first_agent.ipynb -->
<VSCode.Cell language="markdown">
# Chapter 1: Building Your First LangChain Agent

**Purpose:** Learn to create basic and production-ready LangChain agents following official documentation  
**Prerequisites:** Python 3.8+, API keys configured  
**Duration:** 30-45 minutes  
**Key Concepts:** LangChain agents, tools, memory, observability

---

This notebook demonstrates the progression from a basic agent to production-ready systems, following the official LangChain quickstart guide and real-world patterns.
</VSCode.Cell>

<VSCode.Cell language="python">
# Environment Setup and Imports
import sys
import warnings
warnings.filterwarnings('ignore')

# Add project paths
sys.path.append('../../codes/shared')
sys.path.append('../../codes/chapter01')

# Standard LangChain imports
from langchain.agents import create_agent
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage

# Load environment variables and configuration
from dotenv import load_dotenv
load_dotenv()

print("✅ Environment setup complete")
print("📚 LangChain Agent Development - Chapter 1")
</VSCode.Cell>

<VSCode.Cell language="markdown">
## Part 1: Understanding LangChain - The Platform for Agent Engineering

Before building agents, let's understand what LangChain is and why it exists.

> **Quote from LangChain Docs**: "LangChain is the platform for agent engineering. Build agents with any model provider and deploy them anywhere."

LangChain solves the complex infrastructure challenges of building production AI agents by providing:
- **Standard model interface** - works with any LLM provider
- **Pre-built agent architecture** - get started in under 10 lines of code
- **Tool integration** - connect to external systems seamlessly
- **Memory management** - maintain context across conversations
- **Observability** - built-in tracing and monitoring
</VSCode.Cell>

<VSCode.Cell language="python">
# Let's see the difference: Before vs After LangChain

print("🔍 The LangChain Difference")
print("=" * 40)

print("\nBefore LangChain (hundreds of lines needed):")
print("""
class CustomAIAgent:
    def __init__(self):
        self.conversation_history = []    # Manual memory
        self.api_clients = {}             # Custom API integrations  
        self.retry_logic = {}             # Custom error handling
        # ... hundreds more lines of infrastructure code
""")

print("\nWith LangChain (under 10 lines):")
print("""
from langchain.agents import create_agent

agent = create_agent(
    model="gpt-4",
    tools=[weather_tool],
    system_prompt="You are a helpful assistant"
)
""")

print("\n✨ This simplification is the power of LangChain!")
</VSCode.Cell>

<VSCode.Cell language="markdown">
## Part 2: Basic Agent - Following Official LangChain Quickstart

Let's build our first agent using the exact example from the LangChain documentation.

This demonstrates:
1. Simple tool creation
2. Agent instantiation with Claude Sonnet 4.5
3. Basic interaction patterns
</VSCode.Cell>

<VSCode.Cell language="python">
# Basic Agent Implementation (Official LangChain Example)

def get_weather(city: str) -> str:
    """Get weather for a given city."""
    return f"It's always sunny in {city}!"

# Create agent following official quickstart
def create_basic_agent():
    """Create the basic agent from LangChain quickstart."""
    try:
        agent = create_agent(
            model="anthropic:claude-sonnet-4-5",  # Recommended in docs
            tools=[get_weather],
            system_prompt="You are a helpful assistant",
        )
        return agent
    except Exception as e:
        print(f"❌ Error creating agent: {e}")
        print("💡 Make sure ANTHROPIC_API_KEY is set in your environment")
        return None

# Create and test the basic agent
basic_agent = create_basic_agent()

if basic_agent:
    print("✅ Basic agent created successfully!")
    print("🤖 Agent ready for queries")
else:
    print("⚠️ Could not create basic agent - check API key configuration")
</VSCode.Cell>

<VSCode.Cell language="python">
# Test the Basic Agent

if basic_agent:
    print("🧪 Testing Basic LangChain Agent")
    print("=" * 40)
    
    test_queries = [
        "what is the weather in sf",
        "How's the weather in New York?",
        "Tell me about the weather in Tokyo"
    ]
    
    for i, query in enumerate(test_queries, 1):
        print(f"\n🔵 Query {i}: {query}")
        print("-" * 30)
        
        try:
            response = basic_agent.invoke({
                "messages": [{"role": "user", "content": query}]
            })
            
            # Extract response content
            if isinstance(response, dict) and 'messages' in response:
                agent_response = response['messages'][-1]['content']
            else:
                agent_response = str(response)
            
            print(f"🤖 Agent Response: {agent_response}")
            
        except Exception as e:
            print(f"❌ Error: {e}")
    
    print(f"\n✅ Basic agent testing complete!")
else:
    print("⚠️ Skipping basic agent tests - agent not available")
</VSCode.Cell>

<VSCode.Cell language="markdown">
## Part 3: Production-Ready Agent - Real-World Implementation

Now let's build a comprehensive production agent that demonstrates:

1. **Detailed system prompts** for consistent behavior
2. **Runtime context integration** for user-specific data
3. **Structured response formats** for application integration
4. **Conversation memory** for natural dialogue
5. **Advanced tool documentation** for better model understanding

This follows the "real-world agent" example from LangChain documentation.
</VSCode.Cell>

<VSCode.Cell language="python">
# Production Agent Implementation

from langchain.tools import tool, ToolRuntime
from langchain.chat_models import init_chat_model
from langgraph.checkpoint.memory import InMemorySaver
from dataclasses import dataclass
from typing import Any

# System prompt defines agent behavior (specific and actionable)
SYSTEM_PROMPT = """You are an expert weather forecaster, who speaks in puns.

You have access to two tools:

- get_weather_for_location: use this to get the weather for a specific location
- get_user_location: use this to get the user's location

If a user asks you for the weather, make sure you know the location. If you can tell from the question that they mean wherever they are, use the get_user_location tool to find their location."""

@tool
def get_weather_for_location(city: str) -> str:
    """Get weather for a given city."""
    return f"It's always sunny in {city}!"

@dataclass 
class Context:
    """Custom runtime context schema."""
    user_id: str

@tool
def get_user_location(runtime: ToolRuntime[Context, Any]) -> str:
    """Retrieve user information based on user ID."""
    user_id = runtime.context.user_id
    return "Florida" if user_id == "1" else "SF"

@dataclass
class ResponseFormat:
    """Response schema for the agent."""
    # A punny response (always required)
    punny_response: str
    # Any interesting information about the weather if available  
    weather_conditions: str | None = None

print("🔧 Production agent components defined!")
</VSCode.Cell>

<VSCode.Cell language="python">
# Create the Production Agent

def create_production_agent():
    """Create a production-ready agent with all advanced features."""
    try:
        # Configure model with production parameters
        model = init_chat_model(
            "anthropic:claude-sonnet-4-5",
            temperature=0.5,     # Balanced creativity vs consistency
            timeout=10,          # Request timeout in seconds
            max_tokens=1000      # Limit response length
        )
        
        # Add memory for conversation continuity
        checkpointer = InMemorySaver()
        
        # Assemble the complete production agent
        agent = create_agent(
            model=model,
            system_prompt=SYSTEM_PROMPT,
            tools=[get_user_location, get_weather_for_location],
            context_schema=Context,
            response_format=ResponseFormat,
            checkpointer=checkpointer
        )
        
        return agent, checkpointer
        
    except Exception as e:
        print(f"❌ Error creating production agent: {e}")
        return None, None

# Create the production agent
production_agent, checkpointer = create_production_agent()

if production_agent:
    print("✅ Production agent created successfully!")
    print("🚀 Agent ready with advanced features:")
    print("   • Runtime context integration")
    print("   • Structured response format") 
    print("   • Conversation memory")
    print("   • Advanced tool documentation")
else:
    print("⚠️ Could not create production agent")
</VSCode.Cell>

<VSCode.Cell language="python">
# Test the Production Agent

if production_agent:
    print("🧪 Testing Production LangChain Agent")
    print("=" * 50)
    
    # Configure conversation thread and user context
    config = {"configurable": {"thread_id": "notebook_demo"}}
    user_context = Context(user_id="1")
    
    test_scenarios = [
        {
            "name": "Contextual Weather Query",
            "query": "what is the weather outside?",
            "description": "Tests tool usage and context integration"
        },
        {
            "name": "Follow-up Conversation", 
            "query": "thank you!",
            "description": "Tests memory and conversation continuity"
        },
        {
            "name": "Specific Location Query",
            "query": "How's the weather in Paris?", 
            "description": "Tests direct location handling"
        }
    ]
    
    for i, scenario in enumerate(test_scenarios, 1):
        print(f"\n🔷 Scenario {i}: {scenario['name']}")
        print(f"   Description: {scenario['description']}")
        print(f"   Query: {scenario['query']}")
        print("-" * 50)
        
        try:
            response = production_agent.invoke(
                {"messages": [{"role": "user", "content": scenario['query']}]},
                config=config,
                context=user_context
            )
            
            # Extract structured response
            if 'structured_response' in response:
                structured_resp = response['structured_response']
                print(f"🎭 Punny Response: {structured_resp.punny_response}")
                if structured_resp.weather_conditions:
                    print(f"🌤️  Weather Info: {structured_resp.weather_conditions}")
            else:
                print(f"🤖 Response: {response}")
                
        except Exception as e:
            print(f"❌ Error: {e}")
        
        print()
    
    print("✅ Production agent testing complete!")
    
else:
    print("⚠️ Skipping production agent tests - agent not available")
</VSCode.Cell>

<VSCode.Cell language="markdown">
## Part 4: OpenAI Alternative Implementation

For comparison, let's see how to implement similar functionality using OpenAI with more traditional LangChain patterns.

This demonstrates:
- Traditional ReAct agent architecture  
- Manual memory management
- Custom tool creation
- Performance monitoring
</VSCode.Cell>

<VSCode.Cell language="python">
# OpenAI Agent Implementation

from langchain_openai import ChatOpenAI
from langchain.agents import tool, create_react_agent, AgentExecutor
from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import PromptTemplate

@tool
def simple_calculator(expression: str) -> str:
    """Evaluate basic mathematical expressions."""
    try:
        # Safe evaluation for basic math
        result = eval(expression.replace('^', '**'))
        return f"The result is: {result}"
    except Exception as e:
        return f"Error calculating: {str(e)}"

@tool
def get_word_count(text: str) -> str:
    """Count words in a text string."""
    word_count = len(text.split())
    char_count = len(text)
    return f"Text '{text}' has {word_count} words and {char_count} characters."

def create_openai_agent():
    """Create OpenAI-based agent for comparison."""
    try:
        # Create OpenAI LLM
        llm = ChatOpenAI(model="gpt-4", temperature=0.1)
        
        # Define tools
        tools = [simple_calculator, get_word_count]
        
        # Create ReAct prompt
        template = """Answer questions using available tools when needed.

Tools available: {tools}

Use this format:
Question: the input question
Thought: think about what to do  
Action: choose from [{tool_names}]
Action Input: tool input
Observation: tool result
... (repeat as needed)
Thought: I know the final answer
Final Answer: complete answer

Question: {input}
Thought: {agent_scratchpad}"""
        
        prompt = PromptTemplate.from_template(template)
        agent = create_react_agent(llm, tools, prompt)
        
        return AgentExecutor(
            agent=agent,
            tools=tools,
            verbose=True,
            max_iterations=3
        )
        
    except Exception as e:
        print(f"❌ Error creating OpenAI agent: {e}")
        print("💡 Make sure OPENAI_API_KEY is set in your environment")
        return None

# Create OpenAI agent
openai_agent = create_openai_agent()

if openai_agent:
    print("✅ OpenAI agent created successfully!")
    print("🔗 Agent ready with tools: calculator, word_count")
else:
    print("⚠️ Could not create OpenAI agent - check API key")
</VSCode.Cell>

<VSCode.Cell language="python">
# Test OpenAI Agent

if openai_agent:
    print("🧪 Testing OpenAI LangChain Agent")
    print("=" * 40)
    
    test_queries = [
        "What is 15 * 24?",
        "How many words are in 'LangChain is awesome'?",
        "Calculate 100 + 50 and tell me how many words are in 'Hello World'"
    ]
    
    for i, query in enumerate(test_queries, 1):
        print(f"\n🔶 Query {i}: {query}")
        print("-" * 30)
        
        try:
            response = openai_agent.invoke({"input": query})
            print(f"🤖 Final Answer: {response['output']}")
            
        except Exception as e:
            print(f"❌ Error: {e}")
    
    print(f"\n✅ OpenAI agent testing complete!")
    
else:
    print("⚠️ Skipping OpenAI agent tests - agent not available")
</VSCode.Cell>

<VSCode.Cell language="markdown">
## Part 5: Agent Comparison and Best Practices

Let's compare the different approaches and understand when to use each:

### Implementation Comparison

| Aspect | Basic LangChain | OpenAI Implementation | Production Agent |
|--------|----------------|---------------------|------------------|
| **Setup Complexity** | Very Simple (3 lines) | Moderate (15-20 lines) | Complex (50+ lines) |
| **Memory Support** | Built-in with checkpointer | Manual implementation | Advanced with persistence |
| **Structured Output** | Native support | Requires parsing | Full schema validation |
| **Observability** | Automatic tracing | Manual logging | Complete monitoring |
| **Best Use Case** | Quick prototypes | Learning & flexibility | Production systems |

### Key Takeaways

1. **Start Simple**: Begin with basic agents, then add complexity as needed
2. **Use Official Examples**: Follow LangChain documentation for best practices  
3. **Structure Responses**: Define clear output formats for application integration
4. **Add Context**: Use runtime context for personalized experiences
5. **Remember Conversations**: Implement memory for natural dialogue
</VSCode.Cell>

<VSCode.Cell language="python">
# Performance and Feature Comparison

import time

def compare_agent_features():
    """Compare features across different agent implementations."""
    
    print("⚖️ Agent Implementation Comparison")
    print("=" * 50)
    
    comparison_results = {
        "Basic LangChain Agent": {
            "available": basic_agent is not None,
            "features": ["Weather tool", "Claude Sonnet 4.5", "Automatic tracing"],
            "setup_lines": "~5 lines",
            "memory": "Built-in checkpointer",
            "best_for": "Quick prototypes"
        },
        "Production Agent": {
            "available": production_agent is not None, 
            "features": ["Runtime context", "Structured output", "Conversation memory", "Advanced tools"],
            "setup_lines": "~50 lines",
            "memory": "Persistent with InMemorySaver",
            "best_for": "Production systems"
        },
        "OpenAI Agent": {
            "available": openai_agent is not None,
            "features": ["Calculator tool", "Word count", "ReAct reasoning", "Manual memory"],
            "setup_lines": "~20 lines", 
            "memory": "Manual ConversationBufferMemory",
            "best_for": "Learning and flexibility"
        }
    }
    
    for agent_name, details in comparison_results.items():
        status = "✅ Available" if details["available"] else "❌ Not Available"
        print(f"\n🤖 {agent_name}: {status}")
        if details["available"]:
            print(f"   Features: {', '.join(details['features'])}")
            print(f"   Setup: {details['setup_lines']}")
            print(f"   Memory: {details['memory']}")
            print(f"   Best for: {details['best_for']}")
    
    return comparison_results

# Run comparison
comparison_results = compare_agent_features()

# Summary
available_agents = sum(1 for details in comparison_results.values() if details["available"])
print(f"\n📊 Summary: {available_agents}/{len(comparison_results)} agent types successfully created")
</VSCode.Cell>

<VSCode.Cell language="markdown">
## Summary and Next Steps

Congratulations! You've successfully built and tested multiple LangChain agent implementations:

### What You've Learned

✅ **LangChain Fundamentals**: Understanding the platform for agent engineering  
✅ **Basic Agent Creation**: Following official quickstart patterns  
✅ **Production Features**: Runtime context, structured output, memory  
✅ **Alternative Approaches**: OpenAI implementation for comparison  
✅ **Best Practices**: When to use different agent patterns  

### Key Achievements

- Created agents that can **understand context and remember conversations**
- Implemented **tool usage** for external system integration  
- Used **structured responses** for consistent application integration
- Applied **runtime context** for personalized user experiences
- Established **conversation memory** for natural dialogue flow

### Next Steps in Chapter 2: Modular LangChain Architecture Patterns

In the next chapter, you'll explore:
- Advanced chain patterns (parallel, branching, conditional)
- LangChain Expression Language (LCEL) for composable pipelines  
- Custom tool development and integration
- Output parsing strategies for structured data
- Reusable component libraries and design patterns

---

**Practice Exercise**: Try modifying the production agent to:
1. Add a new tool for currency conversion
2. Implement different response formats based on user type
3. Add error handling and retry logic for tool failures
</VSCode.Cell>
````